In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-08-12 23:49:11--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  60.4MB/s    in 0.4s    

2024-08-12 23:49:12 (60.4 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
print(df_books.shape)
print(df_books.dtypes)
print(df_ratings.shape)
print(df_ratings.dtypes)

(271379, 3)
isbn      object
title     object
author    object
dtype: object
(1149780, 3)
user        int32
isbn       object
rating    float32
dtype: object


In [5]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [6]:
duplicates = df_books['isbn'].duplicated().sum()
print(duplicates)

0


In [7]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [8]:
df_ratings = df_ratings.dropna()
print(df_ratings.shape)

(1149780, 3)


In [9]:
def count_rows(df, threshold, column):
  rating_count = (
    df.
    groupby(by = [column])['rating'].
    count().
    reset_index().
    rename(columns = {'rating': 'total_' + column + '_rating'})
  )
  return rating_count

In [10]:
rating_per_user = count_rows(df_ratings, 200, 'user')
rating_per_user.head()

,user,total_user_rating
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2


In [11]:
rating_per_book = count_rows(df_ratings, 100, 'isbn')
rating_per_book.head()

,isbn,total_isbn_rating
0,0330299891,2
1,0375404120,2
2,0586045007,1
3,9022906116,2
4,9032803328,1


In [12]:
# Cleaning the data

df_ratings = df_ratings.merge(rating_per_user, left_on = 'user', right_on = 'user', how = 'left')
df_ratings = df_ratings.merge(rating_per_book, left_on = 'isbn', right_on = 'isbn', how = 'left')

df_filter = df_ratings.loc[(df_ratings['total_user_rating'] >= 200) & (df_ratings['total_isbn_rating'] >= 100)]
df_merged = df_filter.merge(df_books, on = 'isbn')
df_merged = df_merged.drop_duplicates(['title', 'user'])
print(df_merged.head())
print(df_merged.shape)

     user        isbn  rating  total_user_rating  total_isbn_rating  \
0  277427  002542730X    10.0                497                171   
1    3363  002542730X     0.0                901                171   
2   11676  002542730X     6.0              13602                171   
3   12538  002542730X    10.0               1351                171   
4   13552  002542730X     0.0                709                171   

                                               title             author  
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
(49136, 7)


In [13]:
# Building a pivot table

df_pivot = df_merged.groupby(['title', 'user'])['rating'].last().unstack(fill_value = 0)
df_pivot.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Building the Sparse Matrix

sparse_matrix = csr_matrix(df_pivot.values)
print(sparse_matrix)

  (0, 0)	9.0
  (0, 36)	9.0
  (0, 41)	8.0
  (0, 58)	7.0
  (0, 136)	9.0
  (0, 157)	10.0
  (0, 171)	9.0
  (0, 188)	8.0
  (0, 220)	10.0
  (0, 221)	10.0
  (0, 246)	10.0
  (0, 278)	10.0
  (0, 391)	10.0
  (0, 477)	9.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 602)	9.0
  (0, 693)	9.0
  (0, 761)	8.0
  (0, 768)	7.0
  (0, 827)	10.0
  (0, 876)	10.0
  (1, 10)	9.0
  (1, 25)	10.0
  (1, 41)	9.0
  :	:
  (671, 232)	8.0
  (671, 271)	6.0
  (671, 330)	9.0
  (671, 437)	9.0
  (671, 475)	10.0
  (671, 647)	8.0
  (671, 660)	7.0
  (671, 746)	7.0
  (671, 759)	9.0
  (672, 15)	8.0
  (672, 25)	7.0
  (672, 112)	5.0
  (672, 151)	6.0
  (672, 234)	7.0
  (672, 292)	8.0
  (672, 305)	9.0
  (672, 324)	10.0
  (672, 369)	10.0
  (672, 386)	9.0
  (672, 498)	10.0
  (672, 505)	8.0
  (672, 591)	8.0
  (672, 712)	8.0
  (672, 830)	9.0
  (672, 880)	8.0


In [15]:
# Building the model

knn = NearestNeighbors(algorithm = 'auto', metric = 'cosine')
knn.fit(sparse_matrix)

NearestNeighbors(metric='cosine')

In [16]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_index = df_pivot.index.get_loc(book)
  distances, indices = knn.kneighbors(df_pivot.iloc[book_index, :].to_numpy().reshape(1, -1), n_neighbors = 6)
  recommended_books = []
  recommendations = []
  for i in range(0, len(distances.flatten())):
    if i == 0:
      title = df_pivot.index[book_index]
      recommended_books.append(title)
    else:
      title = df_pivot.index[indices.flatten()[i]]
      distance = distances.flatten()[i]
      recommendations.append([title, distance])
  recommended_books.append(recommendations[::-1])
  return recommended_books

In [17]:
books = get_recommends(book = "Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
